## Average Citizens. ETH Analysis

In [1]:
from config import PROJECT_ID, ETH_ANALYSIS_DATASET_NAME, INITIAL_TS, SNAPSHOT_TS, \
    ETH_ANALYSIS_DISTRIBUTION_TABLE_NAME
from utils import drop_table, create_table

DROP_TABLES = True
CREATE_TABLES = True

### Distribution Rule
<table style="text-align: center">
    <thead style="text-align: center">
        <tr>
            <th rowspan=2></th>
            <th colspan=3>Grade</th>
        </tr>
        <tr>
            <th>1</th>
            <th>2</th>
            <th>3</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align: left"> ETH balance > 0.01 ETH and Amount of outgoing tx</td>
            <td> > 3 tx </td>
            <td> > 21 tx </td>
            <td> > 100 tx </td>
        </tr>
        <tr>
            <td style="text-align: left">OR ETH balance</td>
            <td> > 1 ETH</td>
            <td> > 10 ETH</td>
            <td> > 100 ETH</td>
        </tr>
    </tbody>
</table>

In [2]:
min_eth_balance = 0.01
boundary_outgoing_tx = [3, 21, 100]
boundary_eth_balance = [1, 10, 100]

### Drop Tables

In [3]:
if DROP_TABLES:
    drop_table(ETH_ANALYSIS_DISTRIBUTION_TABLE_NAME, dataset_name=ETH_ANALYSIS_DATASET_NAME)

Table eth_analysis:eth_analysis_distribution deleted.


### ETH Balances

In [5]:
query_1 = f'''
    CREATE TABLE `{PROJECT_ID}.{ETH_ANALYSIS_DATASET_NAME}.{ETH_ANALYSIS_DISTRIBUTION_TABLE_NAME}` AS (
        WITH double_entry_book AS (
            -- debits
            SELECT
                to_address AS address,
                value as value
            FROM `bigquery-public-data.crypto_ethereum.traces`
            WHERE block_timestamp >= '{INITIAL_TS}'
              AND block_timestamp <= '{SNAPSHOT_TS}'
              AND to_address is not null
              AND status = 1
              AND (call_type not in ('delegatecall', 'callcode', 'staticcall') OR call_type is null)
            UNION ALL
            -- credits
            SELECT
                from_address AS address,
                -value AS value
            FROM `bigquery-public-data.crypto_ethereum.traces`
            WHERE block_timestamp >= '{INITIAL_TS}'
              AND block_timestamp <= '{SNAPSHOT_TS}'
              AND from_address is not null
              AND status = 1
              AND (call_type not in ('delegatecall', 'callcode', 'staticcall') OR call_type is null)
            UNION ALL
            -- transaction fees debits
            SELECT
                miner AS address,
                sum(cast(receipt_gas_used AS numeric) * cast(gas_price AS numeric)) AS value
            FROM `bigquery-public-data.crypto_ethereum.transactions` AS transactions
            JOIN `bigquery-public-data.crypto_ethereum.blocks` AS blocks ON blocks.number = transactions.block_number
            WHERE block_timestamp >= '{INITIAL_TS}'
              AND block_timestamp <= '{SNAPSHOT_TS}'
            GROUP BY blocks.miner
            UNION ALL
            -- transaction fees credits
            SELECT
                from_address AS address,
                -(cast(receipt_gas_used AS numeric) * cast(gas_price AS numeric)) AS value
            FROM `bigquery-public-data.crypto_ethereum.transactions`
            WHERE block_timestamp >= '{INITIAL_TS}'
              AND block_timestamp <= '{SNAPSHOT_TS}'
        ),
        outgoing_transactions AS (
            SELECT
                from_address as address,
                count(`hash`) as number_outgoing_transactions
            FROM `bigquery-public-data.crypto_ethereum.transactions`
            WHERE block_timestamp >= '{INITIAL_TS}'
              AND block_timestamp <= '{SNAPSHOT_TS}'
            GROUP BY from_address
        )
        SELECT
            CASE
                WHEN eth_balance > {boundary_eth_balance[2]} AND number_outgoing_transactions > {boundary_outgoing_tx[2]} THEN 'ETH Balance  and Amount of Outgoing Transactions'
                WHEN eth_balance > {boundary_eth_balance[2]} THEN 'ETH Balance'
                WHEN number_outgoing_transactions > {boundary_outgoing_tx[2]} THEN 'Amount of Outgoing Transactions'
                WHEN eth_balance > {boundary_eth_balance[1]} AND number_outgoing_transactions > {boundary_outgoing_tx[1]} THEN 'ETH Balance  and Amount of Outgoing Transactions'
                WHEN eth_balance > {boundary_eth_balance[1]} THEN 'ETH Balance'
                WHEN number_outgoing_transactions > {boundary_outgoing_tx[1]}  THEN 'Amount of Outgoing Transactions'
                WHEN eth_balance > {boundary_eth_balance[0]} AND number_outgoing_transactions > {boundary_outgoing_tx[0]} THEN 'ETH Balance  and Amount of Outgoing Transactions'
                WHEN eth_balance > {boundary_eth_balance[0]} THEN 'ETH Balance'
                WHEN number_outgoing_transactions > {boundary_outgoing_tx[0]} THEN 'Amount of Outgoing Transactions'
                ELSE null
            END
            AS subtype,
            CASE
                WHEN eth_balance > {boundary_eth_balance[2]} OR number_outgoing_transactions > {boundary_outgoing_tx[2]} THEN 3
                WHEN eth_balance > {boundary_eth_balance[1]} OR number_outgoing_transactions > {boundary_outgoing_tx[1]} THEN 2
                WHEN eth_balance > {boundary_eth_balance[0]} OR number_outgoing_transactions > {boundary_outgoing_tx[0]} THEN 1
                ELSE null
            END
            AS grade,
            address,
            eth_balance,
            number_outgoing_transactions
        FROM (
            SELECT
                address,
                eth_balance,
                number_outgoing_transactions
            FROM (
                SELECT
                    address,
                    sum(value)/1e18  AS eth_balance
                FROM double_entry_book
                GROUP BY address
                HAVING eth_balance > {min_eth_balance})
            JOIN outgoing_transactions USING (address)
            )
    )
'''

if CREATE_TABLES:
    create_table(query_1, ETH_ANALYSIS_DISTRIBUTION_TABLE_NAME)